<a href="https://colab.research.google.com/github/clairemasse/Coursera_capstone/blob/master/Comparing_Neighborhoods_in_London_%26_New_York_City.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Coursera Capstone Final Project:


### Introduction & Business Problem ###

For my project I will be comparing neighborhoods in two major cities. I lived in New York City for 3 years, the last of which I lived in Bedford-Stuyvesant, Brooklyn. I am hoping to move to London in the next few months, and will need to think about which neighborhood I should live in. So for this project, I decided to compare neighborhoods in both cities to see which neighborhood in London is the most similar to the one I lived in in Brooklyn.

Hopefully, by the end of my analysis, I will be able to narrow it down to a few neighborhoods in London which will make my apartment search easier if I move to London. 

### Data ###

I will be using geographical data available online for the neighborhoods in both cities and then use the geopy package to get latitude and longitude coordinates. 
I will use the Foursquare API in order to examine and compare the venues (restaurants, bars, gyms, stores...) in both cities. I will use k-means clustering to cluster the neighborhoods based on their similarities - I am hoping that by the end of my analysis I'll be able to narrow it down to a few neighborhoods in London within the same cluster as the one I lived in in Brooklyn. 

In [1]:
#Import all necessary libraries

import pandas as pd
import numpy as np
from IPython.display import display
import requests
import json
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from IPython.display import Image
from IPython.core.display import HTML
# transforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed!')

/bin/bash: conda: command not found
Folium installed!


## Introduction/Business Problem: ##

I lived in New York City for  3 years, the last of which I lived in Bedford-Stuyvesant, Brooklyn. I am hoping to move to London in the next few months, and will need to think about which neighborhood I should aim to live in. So for this project, I decided to compare neighborhoods in both cities to see which neighborhood in London is the most similar to the one I lived in in Brooklyn. 

I will be using geographical data available online for both cities as well as the Foursquare API in order to examine and compare the venues (restaurants, bars, gyms, stores...) in different neighborhoods of both cities. 

In [2]:
# Importing Brooklyn neighborhood data
bk = pd.read_html('http://www.brooklynproperty.com/Brooklyn/ZipCodes.htm')[0]

In [3]:
bk

,City Name,Zip Code,City Name.1,Zip Code.1
0,Brooklyn Heights/Cobble Hill,11201,East Flatbush,11203.0
1,Parkville/Bensonhurst,11204,Fort Greene,11205.0
2,Williamsburg/Bedford-Stuyvesant,11206,East New York,11207.0
3,Cypress Hills,11208,Bay Ridge,11209.0
4,Vanderveer/Flatbush,11210,Williamsburg,11211.0
5,Brownsville,11212,Brower Park/Crown Heights,11213.0
6,Bath Beach/Bensonhurst,11214,Park Slope/Windsor Terrace,11215.0
7,Bedford-Stuyvesant,11216,Park Slope/Gowanus,11217.0
8,Kensington/Windsor Terrace,11218,Borough Park,11219.0
9,Sunset Park,11220,Bushwick/Bedford-Stuyvesant,11221.0


In [4]:
#Clean up the data so it's all in one data frame with 2 columns, neighborhood name and zip code

bk_1 = bk.filter(['City Name', 'Zip Code'], axis = 1)
bk_2 = bk.filter(['City Name.1', 'Zip Code.1'], axis = 1)

bk_2.rename(columns = {'City Name.1': 'City Name', 'Zip Code.1': 'Zip Code'}, inplace=True)

bk = bk_1.append(bk_2)

bk.reset_index(inplace=True)
bk.drop(columns = 'index', inplace=True)

bk.dropna(inplace=True)

bk['Zip Code'] = bk['Zip Code'].astype(int)

bk.rename(columns = {'City Name':'Neighborhood', 'Zip Code': 'Post Code'}, inplace= True)
bk.head()

,Neighborhood,Post Code
0,Brooklyn Heights/Cobble Hill,11201
1,Parkville/Bensonhurst,11204
2,Williamsburg/Bedford-Stuyvesant,11206
3,Cypress Hills,11208
4,Vanderveer/Flatbush,11210


In [5]:
# Define lat/lon functions to add columns to dataframe

geolocator = Nominatim(user_agent="foursquare_agent")

def latitude(address):
  location = geolocator.geocode(address)
  lat = location.latitude
  return lat

def longitude(address):
  location = geolocator.geocode(address)
  lon = location.longitude
  return lon


In [6]:
bk['latitude'] = bk['Post Code'].apply(lambda x: latitude(x))
bk['longitude'] = bk['Post Code'].apply(lambda x: longitude(x))

In [7]:
bk

,Neighborhood,Post Code,latitude,longitude
0,Brooklyn Heights/Cobble Hill,11201,40.696131,-73.996413
1,Parkville/Bensonhurst,11204,40.617336,-73.985333
2,Williamsburg/Bedford-Stuyvesant,11206,40.697646,-73.942416
3,Cypress Hills,11208,40.674337,-73.865342
4,Vanderveer/Flatbush,11210,43.022532,3.035715
5,Brownsville,11212,54.689959,25.421529
6,Bath Beach/Bensonhurst,11214,40.596492,-73.999159
7,Bedford-Stuyvesant,11216,54.686545,25.424749
8,Kensington/Windsor Terrace,11218,54.688060,25.426231
9,Sunset Park,11220,43.091901,2.620856


In [8]:
ldn = pd.read_csv('https://raw.githubusercontent.com/clairemasse/Coursera_capstone/master/london_postcodes%20-%20London_postcodes%20-%20Sheet2%20(1)%20(1).csv')

In [9]:
ldn['location'] = ldn['Neighborhood'].apply(lambda x: geolocator.geocode(x))
ldn.dropna(inplace=True)

In [10]:
ldn.reset_index(inplace=True)
ldn

,index,Neighborhood,Post Code,location
0,0,"Whitechapel, Stepney, Mile End",E1,"(Mile End Road, Globe Town, Stepney, London Bo..."
1,1,Wapping,E1W,"(Wapping, London, Greater London, England, E1W..."
2,2,"Bethnal Green, Shoreditch",E2,"(Mulberry Academy Shoreditch, Gosset Street, S..."
3,3,Clapton,E5,"(Clapton, London Borough of Hackney, London, G..."
4,4,"Hackney, Dalston",E8,"(Dalston, London Borough of Hackney, London, G..."
5,5,"Hackney, Homerton",E9,"(Homerton, Sedgwick Street, Victoria Park Vill..."
6,8,"Bloomsbury, Grays Inn",WC1,"(Bloomsbury Square, St Giles, Holborn, London ..."
7,9,"Covent Garden, Holborn, Strand",WC2,"(Strand, St Clement Danes, Covent Garden, City..."
8,10,"Clerkenwell, Finsbury, Barbican",EC1,"(Barbican, Charterhouse Square, Smithfield, Ci..."
9,12,"Monument, Tower Hill, Aldgate",EC3,"(Tower Bridge, Wapping, London Borough of Towe..."


In [11]:
ldn.drop(['index', 'location'],axis=1,inplace=True)

In [12]:
ldn['latitude'] = ldn['Neighborhood'].apply(lambda x:latitude(x))
ldn['longitude'] = ldn['Neighborhood'].apply(lambda x: longitude(x))

In [13]:
ldn

,Neighborhood,Post Code,latitude,longitude
0,"Whitechapel, Stepney, Mile End",E1,51.520435,-0.053122
1,Wapping,E1W,51.505436,-0.058729
2,"Bethnal Green, Shoreditch",E2,51.527087,-0.070016
3,Clapton,E5,51.558602,-0.055878
4,"Hackney, Dalston",E8,51.546117,-0.075679
5,"Hackney, Homerton",E9,51.547079,-0.043121
6,"Bloomsbury, Grays Inn",WC1,51.518768,-0.122730
7,"Covent Garden, Holborn, Strand",WC2,51.512862,-0.114074
8,"Clerkenwell, Finsbury, Barbican",EC1,51.520150,-0.098683
9,"Monument, Tower Hill, Aldgate",EC3,51.505499,-0.075358


In [14]:
neighborhoods = ldn.append(bk)

In [15]:
neighborhoods.reset_index(inplace=True)


In [16]:
neighborhoods.drop('index', axis=1, inplace= True)

In [17]:
neighborhoods

,Neighborhood,Post Code,latitude,longitude
0,"Whitechapel, Stepney, Mile End",E1,51.520435,-0.053122
1,Wapping,E1W,51.505436,-0.058729
2,"Bethnal Green, Shoreditch",E2,51.527087,-0.070016
3,Clapton,E5,51.558602,-0.055878
4,"Hackney, Dalston",E8,51.546117,-0.075679
...,...,...,...,...
85,Midwood,11230,54.682850,25.427139
86,Industry City/Sunset Park,11232,54.681213,25.423904
87,Flatlands/Mill Basin,11234,40.586176,-73.886609
88,Canarsie,11236,40.638751,-73.902877


In [18]:
# create map of Brooklyn using latitude and longitude values
bk_lat = 40.650002
bk_lon = -73.949997
map_bk = folium.Map(location=[bk_lat, bk_lon], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['latitude'], neighborhoods['longitude'], neighborhoods['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bk)  
    
map_bk

In [19]:
# create map of London using latitude and longitude values
ldn_lat = 51.509865
ldn_lon = -0.118092
map_ldn = folium.Map(location=[ldn_lat, ldn_lon], zoom_start=11.5)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['latitude'], neighborhoods['longitude'], neighborhoods['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ldn)  
    
map_ldn

In [20]:
# Now that we have all the neighborhoods in both cities marked and identified, it's time cluster them using Foursquare API location data

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [21]:
# Foursquare API credentials

CLIENT_ID = 'GLELENDKNLLSRPHH3U1MA2UMELUP5XGF2BLSAPNBONU1JNUN' 
CLIENT_SECRET = 'UUMVCYSAIG5SXZR2KAD1WIROGCP5DIEM3N3E4BT1I1QEDSAE' 
VERSION = '20180605' 



In [22]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
bkldn_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['latitude'],
                                   longitudes=neighborhoods['longitude']
                                  )

Whitechapel, Stepney, Mile End
Wapping
Bethnal Green, Shoreditch
Clapton
Hackney, Dalston
Hackney, Homerton
Bloomsbury, Grays Inn
Covent Garden, Holborn, Strand
Clerkenwell, Finsbury, Barbican
Monument, Tower Hill, Aldgate
Islington, Barnsbury, Canonbury
Finsbury Park, Manor House
Highbury
Highgate
Holloway
Hornsey, Crouch End
Seven Sisters
Stoke Newington, Stamford Hill
Archway, Tufnell Park
Regents Park, Camden Town
Hampstead, Swiss Cottage
Kentish Town
St Johns Wood
Willesden, Harlesden, Kensal Green
Brockley, Crofton Park, Honor Oak Park
Camberwell
Lambeth
Lee, Grove Park
New Cross, New Cross Gate
Peckham, Nunhead
Walworth, Elephant & Castle
East Dulwich
Forest Hill
Herne Hill
Westminster, Belgravia, Pimlico
Chelsea, Brompton
Clapham
Earls Court
Fulham, Parsons Green
South Kensington
Stockwell, Brixton
West Brompton, Worlds End
Battersea, Clapham Junction
Balham
Mayfair, Marylebone, Soho
Bayswater, Paddington
Hammersmith
Kensington
Maida Vale, Warwick Avenue
Ladbroke Grove, North K

In [24]:
print('The dataframe returns', bkldn_venues.shape[0], 'venues.')
bkldn_venues.head()

The dataframe returns 3640 venues.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Whitechapel, Stepney, Mile End",51.520435,-0.053122,Rinkoff's Bakery,51.519964,-0.053238,Bakery
1,"Whitechapel, Stepney, Mile End",51.520435,-0.053122,Genesis Cinema,51.521036,-0.051073,Movie Theater
2,"Whitechapel, Stepney, Mile End",51.520435,-0.053122,One Mile End,51.520151,-0.056136,Brewery
3,"Whitechapel, Stepney, Mile End",51.520435,-0.053122,Mouse Tail Coffee Stories,51.519471,-0.058573,Coffee Shop
4,"Whitechapel, Stepney, Mile End",51.520435,-0.053122,Basicsalon,51.520669,-0.056093,Cosmetics Shop


In [25]:
# Number of venues per neighborhood
display(bkldn_venues.groupby('Neighborhood')['Venue'].count().head(10))

Neighborhood
Archway, Tufnell Park           49
Balham                           1
Bath Beach/Bensonhurst          29
Battersea, Clapham Junction     38
Bay Ridge                       14
Bayswater, Paddington           63
Bedford-Stuyvesant               5
Bethnal Green, Shoreditch      100
Bloomsbury, Grays Inn           66
Borough Park                     6
Name: Venue, dtype: int64

In [26]:
print('There are {} unique categories.'.format(len(bkldn_venues['Venue Category'].unique())))

There are 326 unique categories.


We will use one-hot encoding to analyze all the different neighborhoods

In [27]:
# one hot encoding
bkldn_onehot = pd.get_dummies(bkldn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bkldn_onehot['Neighborhood'] = bkldn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bkldn_onehot.columns[-1]] + list(bkldn_onehot.columns[:-1])
bkldn_onehot = bkldn_onehot[fixed_columns]

bkldn_onehot.head()

,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boarding House,...,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Train Station,Turkish Restaurant,Used Bookstore,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Watch Shop,Waterfront,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Xinjiang Restaurant,Yakitori Restaurant,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
#Let's group by neighborhood and take the mean of the frequency of occurence of each category

bkldn_grouped = bkldn_onehot.groupby('Neighborhood').mean().reset_index()
display(bkldn_grouped.head())


,Neighborhood,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,...,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Train Station,Turkish Restaurant,Used Bookstore,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Watch Shop,Waterfront,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Xinjiang Restaurant,Yakitori Restaurant,Yoga Studio
0,"Archway, Tufnell Park",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020408,0.000000,0.040816,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.020408,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040816,0.0,0.0,0.0,0.0,0.000000,0.0,0.020408,0.0,0.0,0.0,0.0,0.000000,0.020408,0.000000,0.0,0.0,0.020408,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Balham,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Bath Beach/Bensonhurst,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.034483,0.000000,0.0,0.0,0.0,0.0,0.034483,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.034483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.034483,0.0,0.000000,0.0,0.0,0.0,0.0,0.034483,0.000000,0.034483,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,"Battersea, Clapham Junction",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052632,0.000000,0.078947,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.026316,0.0,0.0,0.0,0.0,0.0,...,0.026316,0.0,0.0,0.0,0.078947,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.026316,0.026316,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.026316
4,Bay Ridge,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [29]:
#Top 5 venues in each neighborhood

num_top_venues = 5

for hood in bkldn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bkldn_grouped[bkldn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Archway, Tufnell Park----
                  venue  freq
0    Italian Restaurant  0.10
1                   Pub  0.10
2           Music Venue  0.06
3  Ethiopian Restaurant  0.04
4                  Café  0.04


----Balham----
                   venue  freq
0             Watch Shop   1.0
1           Optical Shop   0.0
2  Performing Arts Venue   0.0
3       Pedestrian Plaza   0.0
4            Pastry Shop   0.0


----Bath Beach/Bensonhurst----
                venue  freq
0         Pizza Place  0.14
1                Park  0.07
2  Italian Restaurant  0.07
3         Gas Station  0.07
4  Russian Restaurant  0.03


----Battersea, Clapham Junction----
           venue  freq
0            Bar  0.08
1    Supermarket  0.08
2  Grocery Store  0.08
3       Pharmacy  0.08
4            Pub  0.08


----Bay Ridge----
           venue  freq
0  Garden Center  0.07
1         Garden  0.07
2  Grocery Store  0.07
3     Skate Park  0.07
4           Food  0.07


----Bayswater, Paddington----
                venu

In [30]:
### Put above into a pandas dataframe

# Function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]


# create dataframe and display top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bkldn_grouped['Neighborhood']

for ind in np.arange(bkldn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bkldn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Archway, Tufnell Park",Pub,Italian Restaurant,Music Venue,Ethiopian Restaurant,Thai Restaurant,Café,Pizza Place,Park,Coffee Shop,Gym / Fitness Center
1,Balham,Watch Shop,Yoga Studio,Ethiopian Restaurant,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
2,Bath Beach/Bensonhurst,Pizza Place,Park,Gas Station,Italian Restaurant,Sporting Goods Shop,Beach,Bank,Donut Shop,Russian Restaurant,German Restaurant
3,"Battersea, Clapham Junction",Pub,Grocery Store,Supermarket,Pharmacy,Bar,Coffee Shop,Bakery,Sandwich Place,Juice Bar,Burrito Place
4,Bay Ridge,Shipping Store,Pier,Park,Café,Skate Park,Dog Run,Beach,Grocery Store,Deli / Bodega,Garden Center


CLUSTER USING K-MEANS


In [31]:
# set number of clusters
kclusters = 4

bkldn_grouped_clustering = bkldn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bkldn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 1, 1, 1, 1, 2, 1, 1, 2], dtype=int32)

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bkldn_merged = neighborhoods

# merge bkldn data with original data to add latitude/longitude for each neighborhood
bkldn_merged = bkldn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
bkldn_merged.head() 

,Neighborhood,Post Code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Whitechapel, Stepney, Mile End",E1,51.520435,-0.053122,1.0,Pub,Supermarket,Coffee Shop,Sandwich Place,Brewery,Chinese Restaurant,Turkish Restaurant,Fast Food Restaurant,Burger Joint,Fried Chicken Joint
1,Wapping,E1W,51.505436,-0.058729,1.0,Coffee Shop,Park,Convenience Store,Pub,Italian Restaurant,Event Space,Bar,Liquor Store,Food & Drink Shop,Fast Food Restaurant
2,"Bethnal Green, Shoreditch",E2,51.527087,-0.070016,1.0,Café,Coffee Shop,Pub,Grocery Store,Wine Bar,Italian Restaurant,Turkish Restaurant,Art Gallery,Bagel Shop,Park
3,Clapton,E5,51.558602,-0.055878,1.0,Grocery Store,Pub,Convenience Store,Garden,Café,Beer Bar,Sake Bar,Dumpling Restaurant,Train Station,Turkish Restaurant
4,"Hackney, Dalston",E8,51.546117,-0.075679,1.0,Café,Grocery Store,Pub,Coffee Shop,Cocktail Bar,Bar,Restaurant,Pizza Place,Turkish Restaurant,Jazz Club


In [33]:
bkldn_merged.groupby('Cluster Labels').count()

,Neighborhood,Post Code,latitude,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,
0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1.0,81,81,81,81,81,81,81,81,81,81,81,81,81,81
2.0,6,6,6,6,6,6,6,6,6,6,6,6,6,6
3.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
